In [1]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from test import HierarchicalPRMBase
from test_Lazy import VisPRM_with_LazyPRM
from test import VisPRM_with_BasicPRM
from HierarchicalPRMVisualization import hierarchicalPRMVisualize
from lectures.IPEnvironment import CollisionChecker
from environment import get_all_scenes
import time
import pandas as pd
from scipy.spatial.distance import euclidean

In [10]:
# ---- Configuration for Hierarchical PRM ----
config_hierarchical = {
    "numNodes": 50,               # Number of nodes in main roadmap
    "boundingBoxSize": 2.0,       # Size of local planning regions
    "discardOverconnected": True, # Whether to prune over-connected nodes
    "debug": False,               # Enable detailed logging
    "internalConfig": {           # Configuration passed to internal planner
        "radius": 1.5,            # For BasicPRM
        "numNodes": 20,           # For BasicPRM
        "initialRoadmapSize": 15,  # For LazyPRM
        "updateRoadmapSize": 5,    # For LazyPRM
        "kNearest": 5             # For LazyPRM
    }
}

config = {
    "initialRoadmapSize": 50,
    "updateRoadmapSize": 20,
    "kNearest": 10,
    "maxIterations": 30
}


scenes = get_all_scenes()

In [11]:
# Create directories for results
import os
plot_dir = "plots_hierarchical"
results_dir = "benchmark_results"
os.makedirs(plot_dir, exist_ok=True)
os.makedirs(results_dir, exist_ok=True)

In [12]:
def run_benchmark(planner_class, planner_name):
    all_results = []
    scenes = get_all_scenes()

    for scene_name, (scene, limits, (start_pos, goal_pos)) in scenes:
        start = [list(start_pos)]
        goal = [list(goal_pos)]

        collision_checker = CollisionChecker(scene, limits)
        planner = planner_class(collision_checker)

        t0 = time.time()
        result = planner.planPath(start, goal, config)
        t1 = time.time()

        # Sicheres Extrahieren des Pfades aus dem Ergebnis
        path = result.get('path', []) if isinstance(result, dict) else []

        # Berechnung der Pfadmetriken mit Fehlerprüfung
        try:
            coords = [planner.graph.nodes[node]['pos'] for node in path] if path else []
            euclidean_length = sum(euclidean(a, b) for a, b in zip(coords[:-1], coords[1:])) if coords else 0
        except KeyError:
            print(f"Warnung: Ungültige Knoten-IDs im Pfad für Szene {scene_name}")
            coords = []
            euclidean_length = 0

        result_data = {
            "scene": scene_name,
            "hasPath": bool(path),
            "length": len(path),
            "euclidean_length": euclidean_length,
            "time": t1 - t0,
            "nodes": len(planner.graph.nodes),
            "edges": len(planner.graph.edges)
        }
        all_results.append(result_data)

    return all_results

In [13]:
# ---- Run both variants ----
all_results = []

# LazyPRM version
lazy_results = run_benchmark(VisPRM_with_LazyPRM, "Hierarchical-LazyPRM")
lazy_df = pd.DataFrame(lazy_results)
lazy_df.to_csv(os.path.join(results_dir, "Hierarchical-LazyPRM_results.csv"), index=False)
all_results.extend(lazy_results)
